In [1]:
cd F:\MTP

F:\MTP


In [2]:
import os
import torch
from collections import defaultdict
from sklearn.model_selection import train_test_split
import numpy as np
import tensorflow as tf

In [3]:
import os
path = os.listdir('EmoDB/wav')

We extract speaker embeddings(x-vectors) with a pretrained TDNN model using SpeechBrain from each utternace with a size of 512.

In [4]:
import torchaudio
from speechbrain.pretrained import EncoderClassifier
classifier = EncoderClassifier.from_hparams(source="speechbrain/spkrec-xvect-voxceleb", savedir="pretrained_models/spkrec-xvect-voxceleb")

Each utterance from the database is name according to the scheme :
    Positions 1-2: number of speaker
    Positions 3-5: code for text
    Position 6: emotion (sorry, letter stands for german emotion word)
    Position 7: if there are more than two versions these are numbered a, b, c....

Example: 03a01Fa.wav is the audio file from Speaker 03 speaking text a01 with
the emotion "Freude" (Happiness in German).

So now,we extract the emotion from the filenames and consider it as y in our model.

In [5]:
emotion_code = {
    'W':0, #anger
    'L':1, #boredom
    'E':2, #disgust
    'A':3, #fear
    'F':4, #happy
    'T':5, #sad
    'N':6  #neutral
}

speaker_code = {
    '03':0,
    '08':1,
    '09':2,
    '10':3,
    '11':4,
    '12':5,
    '13':6,
    '14':7,
    '15':8,
    '16':9
}

gender_code = {
    '03':0,
    '08':1,
    '09':1,
    '10':0,
    '11':0,
    '12':0,
    '13':1,
    '14':1,
    '15':0,
    '16':1
}

In [6]:
x_vec_size = 512
X = torch.zeros([len(path),x_vec_size])
y = torch.zeros([len(path),7],dtype = int)
for i in range(len(path)):
    signal, fs =torchaudio.load('EmoDB/wav/'+path[i])
    embeddings = classifier.encode_batch(signal)
    X[i] = embeddings[0][0]
    y[i][emotion_code[path[i][5]]]=1

In [7]:
ys = torch.zeros([len(path),10],dtype = int)
yg = torch.zeros([len(path),2],dtype = int)
for i in range(len(path)):
    ys[i][speaker_code[path[i][:2]]]=1
    yg[i][gender_code[path[i][:2]]]=1

In [8]:
yes = []
for i in range(535):
    yes.append([y[i],ys[i]])

In [9]:
#print(ys[0])

Convert them into numpy arrays so that we can train them easily

In [9]:
X=X.detach().numpy()
y=y.detach().numpy()
#ys=ys.detach().numpy()

In [ ]:
dict_X=defaultdict(list)
dict_y=defaultdict(list)
#dict_ys=defaultdict(list)
for i in range(len(path)):
    dict_X[path[i][:2]].append(X[i])
    dict_y[path[i][:2]].append(y[i])
   # dict_ys[path[i][:2]].append(ys[i])

In [12]:
#print(X.shape,ys.shape)

To make this evaluation speaker independent, we separate the data speaker-wise and use leave-one-out validation.

In [ ]:
dict = {0:"03",1:"08",2:"09",3:"10",4:"11",5:"12",6:"13",7:"14",8:"15",9:"16"}

In [ ]:
#requirements


from keras.models import Sequential
from keras.layers import Dense, Dropout, Activation, Flatten
from keras.layers import Convolution2D, MaxPooling2D
from keras.optimizers import Adam
from keras.utils import np_utils
from sklearn import metrics
from sklearn.metrics import accuracy_score
from keras.callbacks import ModelCheckpoint


Since the x-vectors are extracted before the last layer from a pretrained model, we just add softmax layer to get the output layer.

In [ ]:
def train_and_test(X,y,X_test,y_test,X_valid,y_valid):
    model = Sequential()
    model.add(Dense(7, input_shape=(len(X[0]),)))
    model.add(Activation('softmax'))
    model.compile(loss='categorical_crossentropy', metrics=['accuracy'], optimizer='adam')
    checkpoint_filepath = './tmp/checkpoint'
    model_checkpoint_callback =ModelCheckpoint(filepath=checkpoint_filepath,monitor='val_accuracy',mode='max',save_best_only=True)
    model.fit(X,y,batch_size=20, epochs=200,callbacks=model_checkpoint_callback,validation_data=(X_valid,y_valid))
    loss,acc=model.evaluate(X_test,y_test)
    print(acc)
    return acc

In [ ]:
def train_and_test_speak(X,y,X_test,y_test,X_valid,y_valid):
    model = Sequential()
    model.add(Dense(10, input_shape=(len(X[0]),)))
    model.add(Activation('softmax'))
    model.compile(loss='categorical_crossentropy', metrics=['accuracy'], optimizer='adam')
    checkpoint_filepath = './tmp/checkpoint_speakers'
    model_checkpoint_callback =ModelCheckpoint(filepath=checkpoint_filepath,monitor='val_accuracy',mode='max',save_best_only=True)
    model.fit(X,y,batch_size=20, epochs=200,callbacks=model_checkpoint_callback,validation_data=(X_valid,y_valid))
    loss,acc=model.evaluate(X_test,y_test)
    print(acc)
    return acc

In [ ]:
temp = 9
acc = 0
accuracies = [0]*10
for i in range(10):
    if temp == 0:
        valid = 9
    else:
        valid = temp-1
    
    a = list((dict_X[dict[i]] for i in range(10) if i not in [temp,valid] ))
    b = list((dict_y[dict[i]] for i in range(10) if i not in [temp,valid]))
    X_ = np.concatenate(a)
    X_test = np.array(dict_X[dict[temp]])
    X_valid = np.array(dict_X[dict[valid]])
    y_test = np.array(dict_y[dict[temp]])
    y_valid = np.array(dict_y[dict[valid]])
    y_ = np.concatenate(b)
    temp -= 1
    print(9-temp)
    temp_acc = train_and_test(X_,y_,X_test,y_test,X_valid,y_valid)
    accuracies[temp]=temp_acc
    print(f'Accuracy for iteration: {temp_acc}')
    acc += temp_acc

1
Epoch 1/200
19/21 [==========================>...] - ETA: 0s - loss: 10.3156 - accuracy: 0.2684WARNING:tensorflow:From C:\Users\Pramodh\AppData\Roaming\Python\Python37\site-packages\tensorflow\python\training\tracking\tracking.py:111: Model.state_updates (from tensorflow.python.keras.engine.training) is deprecated and will be removed in a future version.
Instructions for updating:
This property should not be used in TensorFlow 2.0, as updates are applied automatically.
Instructions for updating:
This property should not be used in TensorFlow 2.0, as updates are applied automatically.
INFO:tensorflow:Assets written to: ./tmp\checkpoint\assets
21/21 [==============================] - 2s 106ms/step - loss: 9.8733 - accuracy: 0.2745 - val_loss: 8.5521 - val_accuracy: 0.2143
Epoch 2/200
21/21 [==============================] - 1s 46ms/step - loss: 4.2840 - accuracy: 0.3946 - val_loss: 5.3462 - val_accuracy: 0.3929
Epoch 3/200
21/21 [==============================] - 1s 43ms/step - loss: 2

21/21 [==============================] - 0s 3ms/step - loss: 0.0110 - accuracy: 1.0000 - val_loss: 2.1531 - val_accuracy: 0.6250
Epoch 47/200
21/21 [==============================] - 0s 2ms/step - loss: 0.0107 - accuracy: 1.0000 - val_loss: 2.1812 - val_accuracy: 0.6607
Epoch 48/200
21/21 [==============================] - 0s 3ms/step - loss: 0.0113 - accuracy: 1.0000 - val_loss: 2.1115 - val_accuracy: 0.6250
Epoch 49/200
21/21 [==============================] - 0s 3ms/step - loss: 0.0104 - accuracy: 1.0000 - val_loss: 1.9962 - val_accuracy: 0.6429
Epoch 50/200
21/21 [==============================] - 0s 3ms/step - loss: 0.0100 - accuracy: 1.0000 - val_loss: 2.1250 - val_accuracy: 0.6607
Epoch 51/200
21/21 [==============================] - 0s 3ms/step - loss: 0.0087 - accuracy: 1.0000 - val_loss: 2.1480 - val_accuracy: 0.6429
Epoch 52/200
21/21 [==============================] - 0s 3ms/step - loss: 0.0091 - accuracy: 1.0000 - val_loss: 2.0740 - val_accuracy: 0.6429
Epoch 53/200
21/21 

21/21 [==============================] - 0s 2ms/step - loss: 0.0031 - accuracy: 1.0000 - val_loss: 2.0905 - val_accuracy: 0.6607
Epoch 104/200
21/21 [==============================] - 0s 2ms/step - loss: 0.0030 - accuracy: 1.0000 - val_loss: 2.0616 - val_accuracy: 0.6607
Epoch 105/200
21/21 [==============================] - 0s 3ms/step - loss: 0.0030 - accuracy: 1.0000 - val_loss: 2.0489 - val_accuracy: 0.6607
Epoch 106/200
21/21 [==============================] - 0s 3ms/step - loss: 0.0029 - accuracy: 1.0000 - val_loss: 2.1017 - val_accuracy: 0.6607
Epoch 107/200
21/21 [==============================] - 0s 3ms/step - loss: 0.0029 - accuracy: 1.0000 - val_loss: 2.1650 - val_accuracy: 0.6607
Epoch 108/200
21/21 [==============================] - 0s 3ms/step - loss: 0.0030 - accuracy: 1.0000 - val_loss: 2.1323 - val_accuracy: 0.6607
Epoch 109/200
21/21 [==============================] - 0s 3ms/step - loss: 0.0028 - accuracy: 1.0000 - val_loss: 2.0850 - val_accuracy: 0.6607
Epoch 110/200

21/21 [==============================] - 0s 3ms/step - loss: 0.0014 - accuracy: 1.0000 - val_loss: 1.9274 - val_accuracy: 0.6607
Epoch 161/200
21/21 [==============================] - 0s 3ms/step - loss: 0.0015 - accuracy: 1.0000 - val_loss: 2.0929 - val_accuracy: 0.6607
Epoch 162/200
21/21 [==============================] - 0s 3ms/step - loss: 0.0015 - accuracy: 1.0000 - val_loss: 2.1402 - val_accuracy: 0.6429
Epoch 163/200
21/21 [==============================] - 0s 3ms/step - loss: 0.0013 - accuracy: 1.0000 - val_loss: 1.9840 - val_accuracy: 0.6607
Epoch 164/200
21/21 [==============================] - 0s 3ms/step - loss: 0.0014 - accuracy: 1.0000 - val_loss: 2.0472 - val_accuracy: 0.6607
Epoch 165/200
21/21 [==============================] - 0s 3ms/step - loss: 0.0013 - accuracy: 1.0000 - val_loss: 2.0596 - val_accuracy: 0.6607
Epoch 166/200
21/21 [==============================] - 0s 3ms/step - loss: 0.0012 - accuracy: 1.0000 - val_loss: 1.9492 - val_accuracy: 0.6607
Epoch 167/200

Epoch 11/200
21/21 [==============================] - 0s 6ms/step - loss: 0.5368 - accuracy: 0.8293 - val_loss: 2.0483 - val_accuracy: 0.5797
Epoch 12/200
21/21 [==============================] - 1s 44ms/step - loss: 0.4512 - accuracy: 0.8610 - val_loss: 1.5526 - val_accuracy: 0.7246
Epoch 13/200
21/21 [==============================] - 0s 4ms/step - loss: 0.4581 - accuracy: 0.8561 - val_loss: 1.9278 - val_accuracy: 0.5797
Epoch 14/200
21/21 [==============================] - 0s 4ms/step - loss: 0.3538 - accuracy: 0.8829 - val_loss: 1.5720 - val_accuracy: 0.6812
Epoch 15/200
21/21 [==============================] - 0s 4ms/step - loss: 0.3465 - accuracy: 0.8854 - val_loss: 1.5807 - val_accuracy: 0.6667
Epoch 16/200
21/21 [==============================] - 0s 4ms/step - loss: 0.3314 - accuracy: 0.8683 - val_loss: 1.5834 - val_accuracy: 0.6957
Epoch 17/200
21/21 [==============================] - 0s 4ms/step - loss: 0.2089 - accuracy: 0.9293 - val_loss: 1.7004 - val_accuracy: 0.6377
Epoch

...

Epoch 83/200
22/22 [==============================] - 0s 5ms/step - loss: 0.0057 - accuracy: 1.0000 - val_loss: 0.9887 - val_accuracy: 0.7959
Epoch 84/200
22/22 [==============================] - 0s 3ms/step - loss: 0.0054 - accuracy: 1.0000 - val_loss: 0.8850 - val_accuracy: 0.7959
Epoch 85/200
22/22 [==============================] - 0s 2ms/step - loss: 0.0054 - accuracy: 1.0000 - val_loss: 0.9302 - val_accuracy: 0.7959
Epoch 86/200
22/22 [==============================] - 0s 3ms/step - loss: 0.0052 - accuracy: 1.0000 - val_loss: 0.9651 - val_accuracy: 0.7959
Epoch 87/200
22/22 [==============================] - 0s 3ms/step - loss: 0.0052 - accuracy: 1.0000 - val_loss: 0.9862 - val_accuracy: 0.8163
Epoch 88/200
22/22 [==============================] - 0s 3ms/step - loss: 0.0052 - accuracy: 1.0000 - val_loss: 0.9403 - val_accuracy: 0.8163
Epoch 89/200
22/22 [==============================] - 0s 3ms/step - loss: 0.0051 - accuracy: 1.0000 - val_loss: 0.9598 - val_accuracy: 0.7959
Epoch 

22/22 [==============================] - 0s 2ms/step - loss: 0.0022 - accuracy: 1.0000 - val_loss: 0.8198 - val_accuracy: 0.8163
Epoch 140/200
22/22 [==============================] - 0s 3ms/step - loss: 0.0023 - accuracy: 1.0000 - val_loss: 0.7968 - val_accuracy: 0.8367
Epoch 141/200
22/22 [==============================] - 0s 3ms/step - loss: 0.0022 - accuracy: 1.0000 - val_loss: 0.8152 - val_accuracy: 0.8163
Epoch 142/200
22/22 [==============================] - 0s 3ms/step - loss: 0.0022 - accuracy: 1.0000 - val_loss: 0.7662 - val_accuracy: 0.8367
Epoch 143/200
22/22 [==============================] - 0s 3ms/step - loss: 0.0021 - accuracy: 1.0000 - val_loss: 0.7556 - val_accuracy: 0.8367
Epoch 144/200
22/22 [==============================] - 0s 3ms/step - loss: 0.0022 - accuracy: 1.0000 - val_loss: 0.8408 - val_accuracy: 0.8163
Epoch 145/200
22/22 [==============================] - 0s 3ms/step - loss: 0.0022 - accuracy: 1.0000 - val_loss: 0.8121 - val_accuracy: 0.8163
Epoch 146/200

22/22 [==============================] - 0s 3ms/step - loss: 0.0011 - accuracy: 1.0000 - val_loss: 0.7201 - val_accuracy: 0.8367
Epoch 196/200
22/22 [==============================] - 0s 3ms/step - loss: 0.0011 - accuracy: 1.0000 - val_loss: 0.7394 - val_accuracy: 0.8571
Epoch 197/200
22/22 [==============================] - 0s 3ms/step - loss: 0.0011 - accuracy: 1.0000 - val_loss: 0.7331 - val_accuracy: 0.8367
Epoch 198/200
22/22 [==============================] - 0s 3ms/step - loss: 0.0011 - accuracy: 1.0000 - val_loss: 0.7099 - val_accuracy: 0.8571
Epoch 199/200
22/22 [==============================] - 0s 3ms/step - loss: 0.0011 - accuracy: 1.0000 - val_loss: 0.7262 - val_accuracy: 0.8571
Epoch 200/200
2/2 [==============================] - 0s 4ms/step - loss: 2.4572 - accuracy: 0.6379
0.6379310488700867
Accuracy for iteration: 0.6379310488700867
10
Epoch 1/200
21/21 [==============================] - 1s 50ms/step - loss: 6.3819 - accuracy: 0.3301 - val_loss: 4.4258 - val_accuracy: 

Epoch 43/200
21/21 [==============================] - 0s 3ms/step - loss: 0.0125 - accuracy: 1.0000 - val_loss: 0.5117 - val_accuracy: 0.8169
Epoch 44/200
21/21 [==============================] - 0s 3ms/step - loss: 0.0114 - accuracy: 1.0000 - val_loss: 0.5882 - val_accuracy: 0.8169
Epoch 45/200
21/21 [==============================] - 0s 3ms/step - loss: 0.0123 - accuracy: 1.0000 - val_loss: 0.4505 - val_accuracy: 0.8310
Epoch 46/200
21/21 [==============================] - 0s 3ms/step - loss: 0.0119 - accuracy: 1.0000 - val_loss: 0.4919 - val_accuracy: 0.8310
Epoch 47/200
21/21 [==============================] - 0s 3ms/step - loss: 0.0111 - accuracy: 1.0000 - val_loss: 0.5737 - val_accuracy: 0.8169
Epoch 48/200
21/21 [==============================] - 0s 3ms/step - loss: 0.0103 - accuracy: 1.0000 - val_loss: 0.4810 - val_accuracy: 0.8310
Epoch 49/200
21/21 [==============================] - 0s 3ms/step - loss: 0.0109 - accuracy: 1.0000 - val_loss: 0.4509 - val_accuracy: 0.8310
Epoch 

21/21 [==============================] - 0s 3ms/step - loss: 0.0035 - accuracy: 1.0000 - val_loss: 0.4744 - val_accuracy: 0.8310
Epoch 101/200
21/21 [==============================] - 0s 3ms/step - loss: 0.0033 - accuracy: 1.0000 - val_loss: 0.4134 - val_accuracy: 0.8592
Epoch 102/200
21/21 [==============================] - 0s 3ms/step - loss: 0.0033 - accuracy: 1.0000 - val_loss: 0.4285 - val_accuracy: 0.8592
Epoch 103/200
21/21 [==============================] - 0s 3ms/step - loss: 0.0032 - accuracy: 1.0000 - val_loss: 0.4183 - val_accuracy: 0.8451
Epoch 104/200
21/21 [==============================] - 0s 3ms/step - loss: 0.0034 - accuracy: 1.0000 - val_loss: 0.4962 - val_accuracy: 0.8310
Epoch 105/200
21/21 [==============================] - 0s 3ms/step - loss: 0.0034 - accuracy: 1.0000 - val_loss: 0.4367 - val_accuracy: 0.8592
Epoch 106/200
21/21 [==============================] - 0s 3ms/step - loss: 0.0032 - accuracy: 1.0000 - val_loss: 0.4530 - val_accuracy: 0.8592
Epoch 107/200

leave-one-speaker-out validation accuracies

In [ ]:
accuracies

In [ ]:
print("Overall accuracy :"+ str(acc*10)+"%" )

In [23]:
#model for speaker recognition
# Xtrain1, Xtest, ytrain1, ytest = train_test_split(X.numpy(), ys.numpy(), test_size=0.3, random_state=42)
# Xtrain, Xvalid, ytrain, yvalid = train_test_split(Xtrain1, ytrain1, test_size=0.15, random_state=42)
# model = Sequential()
# model.add(Dense(10, input_shape=(len(X[0]),)))
# model.add(Activation('softmax'))
# model.compile(loss='categorical_crossentropy', metrics=['accuracy'], optimizer='adam')
# checkpoint_filepath = './tmp/checkpoint_speakers'
# model_checkpoint_callback =ModelCheckpoint(filepath=checkpoint_filepath,monitor='val_accuracy',mode='max',save_best_only=True)
# model.fit(Xtrain,ytrain,batch_size=20, epochs=200,callbacks=model_checkpoint_callback,validation_data=(Xvalid,yvalid))
# loss,acc=model.evaluate(Xtest,ytest)
# print(acc)

In [24]:
# model for gender recognition
# Xtrain1, Xtest, ytrain1, ytest = train_test_split(X.numpy(), yg.numpy(), test_size=0.3, random_state=42)
# Xtrain, Xvalid, ytrain, yvalid = train_test_split(Xtrain1, ytrain1, test_size=0.15, random_state=42)
# model = Sequential()
# model.add(Dense(2, input_shape=(len(X[0]),)))
# model.add(Activation('softmax'))
# model.compile(loss='categorical_crossentropy', metrics=['accuracy'], optimizer='adam')
# checkpoint_filepath = './tmp/checkpoint_speakers'
# model_checkpoint_callback =ModelCheckpoint(filepath=checkpoint_filepath,monitor='val_accuracy',mode='max',save_best_only=True)
# model.fit(Xtrain,ytrain,batch_size=20, epochs=200,callbacks=model_checkpoint_callback,validation_data=(Xvalid,yvalid))
# loss,acc=model.evaluate(Xtest,ytest)
# print(acc)


In [44]:
M31_X,M31_y=np.array(dict_X["03"]),np.array(dict_y["03"])
F34_X,F34_y=np.array(dict_X["08"]),np.array(dict_y["08"])
F21_X,F21_y=np.array(dict_X["09"]),np.array(dict_y["09"])
M32_X,M32_y=np.array(dict_X["10"]),np.array(dict_y["10"])
M26_X,M26_y=np.array(dict_X["11"]),np.array(dict_y["11"])
M30_X,M30_y=np.array(dict_X["12"]),np.array(dict_y["12"])
F32_X,F32_y=np.array(dict_X["13"]),np.array(dict_y["13"])
F35_X,F35_y=np.array(dict_X["14"]),np.array(dict_y["14"])
M25_X,M25_y=np.array(dict_X["15"]),np.array(dict_y["15"])
F31_X,F31_y=np.array(dict_X["16"]),np.array(dict_y["16"])

In [ ]:
import librosa

In [ ]:
melspecs =[0 for i in range(len(path))]
Y = torch.zeros([len(path),7],dtype = int)
for i in range(len(path)):
    r, sr = librosa.load('EmoDB/wav/'+path[i], res_type='kaiser_fast')
    melspec = librosa.feature.melspectrogram(y=r, sr=sr,n_fft = 512, hop_length=160, win_length=320,n_mels=24)
    melspecs[i]=melspec
    Y[i][emotion_code[path[i][5]]]=1

In [ ]:
testfeats_X=defaultdict(list)
testfeats_y=defaultdict(list)
testfeats_ys = defaultdict(list)
for i in range(len(path)):
    testfeats_X[path[i][:2]].append(melspecs[i])
    testfeats_y[path[i][:2]].append(Y[i])
    testfeats_ys[path[i][:2]].append(ys[i])

In [ ]:
trainfeats_X=defaultdict(list)
trainfeats_y=defaultdict(list)
trainfeats_ys=defaultdict(list)
for j in range(len(path)):    
    for i in range(0,len(melspecs[j][1]),50):
        temp = melspecs[0][:,i:i+100]
        if len(temp[1])==100:
            trainfeats_X[path[j][:2]].append(melspecs[0][:,i:i+100])
            trainfeats_y[path[j][:2]].append(Y[j])
            trainfeats_ys[path[j][:2]].append(ys[i]) 

In [ ]:
train_set=[]
for j in range(10):    
    for i in range(0,len(trainfeats_X[dict[j]])):
        train_set.append((torch.tensor(trainfeats_X[dict[j]][i].transpose()),trainfeats_y[dict[j]][i]))

In [ ]:
trainfeats_X[dict[j]][i].transpose().shape

In [ ]:
test_set = [] 
for j in range(10):
    for i in range(0,len(testfeats_X[dict[j]])):
        test_set.append((torch.tensor(testfeats_X[dict[j]][i].transpose()),testfeats_y[dict[j]][i]))

In [ ]:
for i in range(10):
    print(torch.tensor(train_set[i][0]).shape)

In [ ]:
for i in range(10):
    print(torch.tensor(test_set[i][0]).shape)

In [ ]:
import random
random.shuffle(train_set)

In [ ]:
def reshapex(x):  
    return torch.reshape(x,(1,x.shape[0],24))
def reshapey(x):
    return torch.reshape(x,(1,7))
def reshapeys(x):
    return torch.reshape(x,(1,10))

In [ ]:
import torch  
import torch.nn as nn
# import torchvision.datasets as datasets
# import torchvision.transforms as transforms
import speechbrain as sb
from speechbrain.nnet.pooling import StatisticsPooling
from speechbrain.nnet.CNN import Conv1d
from speechbrain.nnet.linear import Linear
from speechbrain.nnet.activations import Softmax
from speechbrain.nnet.normalization import BatchNorm1d

class Xvector(torch.nn.Module):
    """This model extracts X-vectors for speaker recognition and diarization.
    Arguments
    ---------
    device : str
        Device used e.g. "cpu" or "cuda".
    activation : torch class
        A class for constructing the activation layers.
    tdnn_blocks : int
        Number of time-delay neural (TDNN) layers.
    tdnn_channels : list of ints
        Output channels for TDNN layer.
    tdnn_kernel_sizes : list of ints
        List of kernel sizes for each TDNN layer.
    tdnn_dilations : list of ints
        List of dilations for kernels in each TDNN layer.
    lin_neurons : int
        Number of neurons in linear layers.
    Example
    -------
    >>> compute_xvect = Xvector('cpu')
    >>> input_feats = torch.rand([5, 10, 40])
    >>> outputs = compute_xvect(input_feats)
    >>> outputs.shape
    torch.Size([5, 1, 512])
    """
    
    def __init__(
        self,
        device,
        activation,
        tdnn_blocks,
        tdnn_channels,
        tdnn_kernel_sizes,
        tdnn_dilations,
        lin_neurons,
        in_channels,
    ):
        
        super().__init__()
        self.blocks = nn.ModuleList()

        # TDNN layers
        for block_index in range(tdnn_blocks):
            out_channels = tdnn_channels[block_index]
            self.blocks.extend(
                [
                    Conv1d(
                        in_channels=in_channels,
                        out_channels=out_channels,
                        kernel_size=tdnn_kernel_sizes[block_index],
                        dilation=tdnn_dilations[block_index],
                    ),
                    activation(),
                    BatchNorm1d(input_size=out_channels),
                ]
            )
            in_channels = tdnn_channels[block_index]

        # Statistical pooling
        self.blocks.append(StatisticsPooling())

        # Final linear transformation
        self.blocks.append(
            Linear(
                input_size=out_channels * 2,
                n_neurons=lin_neurons,
                bias=True,
                combine_dims=False,
            )
        )
        self.blocks.append(Linear(7,input_size=lin_neurons))
#         self.blocks.append(Softmax())
        
        

    def forward(self, x, lens=None):
        """Returns the x-vectors.
        Arguments
        ---------
        x : torch.Tensor
        """

        for layer in self.blocks:
            try:
                x = layer(x, lengths=lens)
            except TypeError:
                x = layer(x)
        return x


In [ ]:
compute_xvect = Xvector("cpu",torch.nn.LeakyReLU,5,[512, 512, 512, 512, 1500],[5, 3, 3, 1, 1],[1, 2, 3, 1, 1],512,24)

In [ ]:
input_feats = [1,265,24]

In [ ]:
input = torch.rand(input_feats)

In [ ]:
output = compute_xvect(input)

In [ ]:
output[0]

In [ ]:
import torch.optim as optim

In [ ]:
# valid_set = train_set[1840:]

In [ ]:
def train_valid(train_set,valid_set):
    optimizer = optim.Adam(compute_xvect.parameters(), lr = 0.0001)
    criterion = nn.CrossEntropyLoss()
    epochs = 5
    min_validloss = np.inf
    
    for epoch in range(epochs):
        print(epoch)
        
        train_loss=0
        
        for data in train_set:    
            X, y = data
            compute_xvect.zero_grad()
            output = compute_xvect(reshapex(X))
            y=reshapey(y)
            loss=criterion(output[0], torch.argmax(y).view(-1))
            loss.backward()
            optimizer.step()
            train_loss+=loss.item()
            
        valid_loss = 0
        
        for data in valid_set:
            X, y = data
            output = compute_xvect(reshapex(X))
            y=reshapey(y)
            loss=criterion(output[0], torch.argmax(y).view(-1))
            valid_loss+=loss.item()

        if min_validloss>valid_loss:
            min_validloss = valid_loss  
            torch.save(compute_xvect.state_dict(), 'saved_model.pth')

In [ ]:
# compute_xvect.eval()

In [ ]:
def test(test_set):
    corr = 0
    total = 0
    with torch.no_grad():
        
        for data in test_set:
            X,y=data
            output = compute_xvect(reshapex(X))
            for idx,i in enumerate(output):
                if torch.argmax(i)==y[idx]:
                    corr+=1
                total+=1
    return corr*100/total
    print("Accuracy = "+str(corr*100/total))

In [79]:
temp = 9
acc = 0
accuracies = [0]*10
for k in range(10):
    if temp == 0:
        valid = 9
    else:
        valid = temp-1
    train_set = []
    test_set  = []
    valid_set = []
    print(valid,temp)
    for j in range(10):
        if j not in [temp,valid]:
            for i in range(0,len(trainfeats_X[dict[j]])):
                train_set.append((torch.tensor(trainfeats_X[dict[j]][i].transpose()),trainfeats_y[dict[j]][i]))
    for i in range(0,len(trainfeats_X[dict[valid]])):
        valid_set.append((torch.tensor(trainfeats_X[dict[valid]][i].transpose()),trainfeats_y[dict[valid]][i]))
    for i in range(0,len(testfeats_X[dict[temp]])):
        test_set.append((torch.tensor(testfeats_X[dict[temp]][i].transpose()),testfeats_y[dict[temp]][i]))
    train_valid(train_set,valid_set)
    temp_acc = test(test_set)
    accuracies[temp]=temp_acc
    print(f'Accuracy for iteration: {temp_acc}')
    acc += temp_acc
    temp-=1

8 9
0
1
2
3
4
Accuracy for iteration: 80.28169014084507
7 8
0
1
2
3
4
Accuracy for iteration: 76.78571428571429
6 7
0
1
2
3
4
Accuracy for iteration: 76.81159420289855
5 6
0
1
2
3
4
Accuracy for iteration: 80.32786885245902
4 5
0
1
2
3
4
Accuracy for iteration: 65.71428571428571
3 4
0
1
2
3
4
Accuracy for iteration: 80.0
2 3
0
1
2
3
4
Accuracy for iteration: 73.6842105263158
1 2
0
1
2
3
4
Accuracy for iteration: 69.76744186046511
0 1
0
1
2
3
4
Accuracy for iteration: 79.3103448275862
9 0
0
1
2
3
4
Accuracy for iteration: 71.42857142857143


In [80]:
accuracies

[71.42857142857143,
 79.3103448275862,
 69.76744186046511,
 73.6842105263158,
 80.0,
 65.71428571428571,
 80.32786885245902,
 76.81159420289855,
 76.78571428571429,
 80.28169014084507]

In [83]:
sum(accuracies)/10

75.41117218391412

In [ ]:
test_set = []
temp=0
for j in range (10):
    for i in range(0,len(testfeats_X[dict[temp]])):
        test_set.append((torch.tensor(testfeats_X[dict[temp]][i].transpose()),testfeats_y[dict[temp]][i],testfeats_ys[dict[temp]][i]))
    temp+=1
random.shuffle(test_set)

In [ ]:
train_set = []
temp=0
for j in range (10):
    for i in range(0,len(trainfeats_X[dict[temp]])):
        train_set.append((torch.tensor(trainfeats_X[dict[temp]][i].transpose()),trainfeats_y[dict[temp]][i],trainfeats_ys[dict[temp]][i]))
    temp+=1
random.shuffle(train_set)

In [ ]:
import torch  
import torch.nn as nn
# import torchvision.datasets as datasets
# import torchvision.transforms as transforms
import speechbrain as sb
from speechbrain.nnet.pooling import StatisticsPooling
from speechbrain.nnet.CNN import Conv1d
from speechbrain.nnet.linear import Linear
from speechbrain.nnet.activations import Softmax
from speechbrain.nnet.normalization import BatchNorm1d

class Xvector(torch.nn.Module):
    """This model extracts X-vectors for speaker recognition and diarization.
    Arguments
    ---------
    device : str
        Device used e.g. "cpu" or "cuda".
    activation : torch class
        A class for constructing the activation layers.
    tdnn_blocks : int
        Number of time-delay neural (TDNN) layers.
    tdnn_channels : list of ints
        Output channels for TDNN layer.
    tdnn_kernel_sizes : list of ints
        List of kernel sizes for each TDNN layer.
    tdnn_dilations : list of ints
        List of dilations for kernels in each TDNN layer.
    lin_neurons : int
        Number of neurons in linear layers.
    Example
    -------
    >>> compute_xvect = Xvector('cpu')
    >>> input_feats = torch.rand([5, 10, 40])
    >>> outputs = compute_xvect(input_feats)
    >>> outputs.shape
    torch.Size([5, 1, 512])
    """
    
    def __init__(
        self,
        device,
        activation,
        tdnn_blocks,
        tdnn_channels,
        tdnn_kernel_sizes,
        tdnn_dilations,
        lin_neurons,
        in_channels,
    ):
        
        super().__init__()
        self.blocks = nn.ModuleList()

        # TDNN layers
        for block_index in range(tdnn_blocks):
            out_channels = tdnn_channels[block_index]
            self.blocks.extend(
                [
                    Conv1d(
                        in_channels=in_channels,
                        out_channels=out_channels,
                        kernel_size=tdnn_kernel_sizes[block_index],
                        dilation=tdnn_dilations[block_index],
                    ),
                    activation(),
                    BatchNorm1d(input_size=out_channels),
                ]
            )
            in_channels = tdnn_channels[block_index]

        # Statistical pooling
        self.blocks.append(StatisticsPooling())

        # Final linear transformation
        self.blocks.append(
            Linear(
                input_size=out_channels * 2,
                n_neurons=lin_neurons,
                bias=True,
                combine_dims=False,
            )
        )
        

    def forward(self, x, lens=None):
        """Returns the x-vectors.
        Arguments
        ---------
        x : torch.Tensor
        """

        for layer in self.blocks:
            try:
                x = layer(x, lengths=lens)
            except TypeError:
                x = layer(x)
        last_em = Linear(7,input_size=512)
        last_sp = Linear(10,input_size=512)
        xem=last_em(x)
        xsp=last_sp(x)
        
        return xem,xsp


In [90]:
model = Xvector("cpu",torch.nn.LeakyReLU,5,[512, 512, 512, 512, 1500],[5, 3, 3, 1, 1],[1, 2, 3, 1, 1],512,24)

In [98]:
input_feats = [1,265,24]
input = torch.rand(input_feats)
outputs = model(input)

In [102]:
outputs[1][0]

tensor([[ 0.0680, -0.3098,  0.0145, -0.2123,  0.1018, -0.4053,  0.0903, -0.0791,
         -0.3930,  0.0397]], grad_fn=<SelectBackward>)

In [101]:
output[0]

tensor([[-0.0644, -0.1593, -0.2593, -0.0735, -0.1632,  0.3316, -0.2480]],
       grad_fn=<SelectBackward>)

In [141]:
def train_valid_mt(train_set,valid_set):
    optimizer = optim.Adam(compute_xvect.parameters(), lr = 0.0001)
    criterion = nn.CrossEntropyLoss()
    epochs = 5
    min_validloss = np.inf
    
    for epoch in range(epochs):
        print(epoch)
        
        train_loss=0
        
        for data in train_set:    
            X, ye, ys = data
            model.zero_grad()
            output = model(reshapex(X))
            ye=reshapey(ye)
            losse=criterion(output[0][0], torch.argmax(ye).view(-1))
            ys = reshapeys(ys)
            lossp = criterion(output[1][0],torch.argmax(ys).view(-1))
            loss = losse + lossp
            loss.backward()
            optimizer.step()
            train_loss+=loss.item()
            
        valid_loss = 0
        
        for data in valid_set:
            X, ye, ys = data
            model.zero_grad()
            output = model(reshapex(X))
            ye=reshapey(ye)
            losse=criterion(output[0][0], torch.argmax(ye).view(-1))
            ys = reshapeys(ys)
            lossp = criterion(output[1][0],torch.argmax(ys).view(-1))
            valid_loss = losse + lossp

        if min_validloss>valid_loss:
            min_validloss = valid_loss  
            torch.save(compute_xvect.state_dict(), 'saved_model.pth')

In [146]:
def test_mt(test_set):
    corr1 = 0
    corr2 = 0
    total = 0
    with torch.no_grad():
        
        for data in test_set:
            X,ye,ys =data
            outputs = model(reshapex(X))
            for idx,i in enumerate(outputs[0]):
                if torch.argmax(i)==ye[idx]:
                    corr1+=1
            for idx,i in enumerate(outputs[1]):
                if torch.argmax(i)==ys[idx]:
                    corr2+=1
                total+=1
    print(corr1,corr2)
    return corr1*100/total,corr2*100/total

In [ ]:
len(train_set)

In [150]:
random.shuffle(test_set)

In [180]:
train_valid_mt(train_set[:1840],train_set[1840:2140])
print(test_mt(test_set))

0
1
2
3
4
59 61
(11.02803738317757, 11.401869158878505)


In [149]:
print(test_mt(test_set[428:535]))

14 13
(13.08411214953271, 12.149532710280374)


In [126]:
y[0] #emotion recognition

array([0, 0, 0, 0, 1, 0, 0], dtype=int64)

In [127]:
outputs #(emotion,speaker)

(tensor([[[ 0.0226, -0.0349, -0.1348,  0.0117,  0.3038, -0.1282,  0.0640]]],
        grad_fn=<AddBackward0>),
 tensor([[[ 0.0680, -0.3098,  0.0145, -0.2123,  0.1018, -0.4053,  0.0903,
           -0.0791, -0.3930,  0.0397]]], grad_fn=<AddBackward0>))

In [131]:
ys[0] #speaker recognition

tensor([1, 0, 0, 0, 0, 0, 0, 0, 0, 0])